In [15]:
import numpy as np 
from scipy.optimize import least_squares
import matplotlib.pyplot as plt
# Function for the distance between two points
def distance(p1, p2):
    return np.linalg.norm(p2 - p1)

# Rotation matrix for small angle approximation
def rotation_matrix(eta):
    return np.array([[1, eta],
                     [-eta, 1]])

# Apply transformation to a point
def transform(p, R, T):
    return np.dot(R, (p + T))

def plot_layers(l2_p1, l2_p2, l2_p3, eta = 0, u=0 , w=0, name='layer'):

	plt.clf()
	origin = np.array([0, 0]) # (x,z)
	solid_length = 400
	solid_width = 200

	# Layer 1 points
	l1_p1 = np.array([-75,60])
	l1_p2 = np.array([-75,-60])
	l1_p3 = np.array([-150,55])

	# Rectangle points
	rect_points = np.array([
		[-solid_length/2, -solid_width/2],
		[solid_length/2, -solid_width/2],
		[solid_length/2, solid_width/2],
		[-solid_length/2, solid_width/2],
		[-solid_length/2, -solid_width/2]
	])

	R = rotation_matrix(eta)
	T = np.array([u, w])

	# Apply transformation to rectangle points
	transformed_rect_points = np.array([transform(p, R, T) for p in rect_points])
	original_rect_points = np.array([transform(p, np.eye(2), np.array([0, 0])) for p in rect_points])
	
	plt.figure(figsize=(6, 10),dpi=100)
	fig, ax = plt.subplots()

	ax.plot(transformed_rect_points[:, 0], transformed_rect_points[:, 1], 'b-')
	ax.plot(original_rect_points[:, 0], original_rect_points[:, 1], 'r-',alpha=0.5)
	plt.plot([0,0],[-200,200],'--k',alpha=0.5,linewidth=0.5)
	plt.plot([-400,400],[0,0],'--k',alpha=0.5,linewidth=0.5)


	ax.set_xlim(-300, 300)
	ax.set_ylim(-180, 180)
	ax.set_aspect('equal', 'box')
	ax.invert_xaxis()

	ax.plot(*origin, 'k+')  # Origin

	ax.plot(*l1_p1, 'yx')   # Layer 1 points
	ax.text(*l1_p1, f'l1_p1', fontsize=5, ha='left', va='top')
	ax.plot(*l1_p2, 'gx')
	ax.text(*l1_p2, f'l1_p2', fontsize=5, ha='left', va='top')
	ax.plot(*l1_p3, 'bx')
	ax.text(*l1_p3, f'l1_p3', fontsize=5, ha='left', va='top')

	ax.plot(*l2_p1, 'yo')   # Layer 2 points
	ax.text(*l2_p1, f'l2_p1', fontsize=5, ha='left', va='top')
	ax.plot(*l2_p2, 'go')
	ax.text(*l2_p2, f'l2_p2', fontsize=5, ha='left', va='top')
	ax.plot(*l2_p3, 'bo')
	ax.text(*l2_p3, f'l2_p3', fontsize=5, ha='left', va='top')

	plt.text(0.03, 0.95, rf'$\Delta \eta$ = {eta:.5f} ', fontsize=8, ha='left', va='top', transform=ax.transAxes)
	plt.text(0.03, 0.9, rf'$\Delta x$ = {u:.5f}', fontsize=8, ha='left', va='top', transform=ax.transAxes)
	plt.text(0.03, 0.85, rf'$\Delta z$ = {w:.5f}', fontsize=8, ha='left', va='top', transform=ax.transAxes)
	ax.set_ylabel('z')
	ax.set_xlabel('x')
	plt.savefig(f'{name}.pdf',bbox_inches='tight')
	plt.show()


def solve(values):
	def residuals(vars):
		u, w, eta = vars

		R = rotation_matrix(eta)
		T = np.array([u, w])

		# Points in layer 1
		p11 = np.array([values['p11x'], values['p11z']])
		p12 = np.array([values['p12x'], values['p12z']])
		p13 = np.array([values['p13x'], values['p13z']])

		# Initial points in layer 2
		p21_0 = np.array([values['p21x_0'], values['p21z_0']])
		p22_0 = np.array([values['p22x_0'], values['p22z_0']])
		p23_0 = np.array([values['p23x_0'], values['p23z_0']])

		# Transformed points in layer 2
		p21_i = transform(p21_0, R, T)
		p22_i = transform(p22_0, R, T)
		p23_i = transform(p23_0, R, T)

		# Calculate distances
		d1_0 = distance(p11, p21_0) #before
		d1_i = distance(p11, p21_i) #after

		d2_0 = distance(p12, p22_0)
		d2_i = distance(p12, p22_i)

		d3_0 = distance(p13, p23_0)
		d3_i = distance(p13, p23_i)

		# Constraints 

		# eq1, eq2, eq3 are the equations for the distances between the turnbuckles attached in layer 1 and layer 2
		eq1 = d1_0 - d1_i
		eq2 = d2_0 - d2_i
		eq3 = d3_0 - d3_i

		# dist_1 is the distance inc/dec in the first turnbuckle 
		# dist_2 is the distance inc/dec in the second turnbuckle
		# dist_3 is the distance inc/dec in the third turnbuckle
		# if the distance is 0, then the corresponding eq is not changed
		if values['dist_1']!=0:
			eq1 = d1_0 + values['dist_1'] - d1_i

		if values['dist_2']!=0:
			eq2 = d2_0 + values['dist_2'] - d2_i

		if values['dist_3']!=0:
			eq3 = d3_0 + values['dist_3'] - d3_i
			
		# eq4, eq5, eq6 are the equations for the distances between the turnbuckles points in layer 2
		d12_0 = distance(p21_0, p22_0)
		d12_i = distance(p21_i, p22_i)

		d23_0 = distance(p22_0, p23_0)
		d23_i = distance(p22_i, p23_i)

		d13_0 = distance(p21_0, p23_0)
		d13_i = distance(p21_i, p23_i)

		# they are constant even after the transformation
		eq4 = d12_0 - d12_i
		eq5 = d23_0 - d23_i
		eq6 = d13_0 - d13_i

		# all the equations are returned to the least_squares function
		return [eq1, eq2, eq3, eq4, eq5, eq6]
	
	# define bounds for (u, w, eta) the translation should not go beyond 3 mm
	bounds = ([-3, -3, -np.inf], [3, 3, np.inf])
	initial_guess = [0.0, 0.0, 0.0]

	# This gives the best u,w,eta for the given constraints
	solution = least_squares(residuals, initial_guess, bounds=bounds)
	u, w, eta = solution.x

	return eta, u, w

def rotate_screw(rotation_1,rotation_2,rotation_3):

	# Distance changed by one rotation of turnbuckle
	delta_dist = 0.25

	# Initial points in layer 2
	p21_0 = np.array([30, 60])
	p22_0 = np.array([30, -60])
	p23_0 = np.array([-150, -50])

	# Values for the constraints
	values = {
		'p11x': -75, 'p11z': 60, 					# x,z coordinates of the first turnbuckle in layer 1
		'p12x': -75, 'p12z': -60,					# x,z coordinates of the second turnbuckle in layer 1
		'p13x': -150, 'p13z': 55,					# x,z coordinates of the third turnbuckle in layer 1
		'p21x_0': p21_0[0], 'p21z_0': p21_0[1],		# x,z coordinates of the first turnbuckle in layer 2
		'p22x_0': p22_0[0], 'p22z_0': p22_0[1],	 	# x,z coordinates of the second turnbuckle in layer 2
		'p23x_0': p23_0[0], 'p23z_0': p23_0[1], 	# x,z coordinates of the third turnbuckle in layer 2
		'dist_1': rotation_1 * delta_dist,			# distance inc/dec in the first turnbuckle
		'dist_2': rotation_2 * delta_dist,			# distance inc/dec in the second turnbuckle
		'dist_3': rotation_3 * delta_dist			# distance inc/dec in the third turnbuckle
	}

	eta, u, w = solve(values)

	# after the values are obtained, the final points in layer 2 are calculated and plotted
	R = rotation_matrix(eta)
	T = np.array([u, w])
	p21_i = transform(np.array([values['p21x_0'], values['p21z_0']]), R, T)
	p22_i = transform(np.array([values['p22x_0'], values['p22z_0']]), R, T)
	p23_i = transform(np.array([values['p23x_0'], values['p23z_0']]), R, T)
	
	# plot the final points	
	plot_layers(p21_i, p22_i, p23_i, eta, u, w)	



In [16]:
from ipywidgets import interactive
import ipywidgets as widgets


slider_rotations_1 = (-50, 50, 1)  # Slider for turnbuckle 1
slider_rotations_2 = (-50, 50, 1)  # Slider for turnbuckle 2
slider_rotations_3 = (-50, 50, 1)  # Slider for turnbuckle 3
reset_button = widgets.Button(description="Reset")

def reset_sliders(b):
	interactive_plot.children[0].value = 0
	interactive_plot.children[1].value = 0
	interactive_plot.children[2].value = 0

reset_button.on_click(reset_sliders)
display(reset_button)
p21_0 = np.array([30, 60])
p22_0 = np.array([30, -60])
p23_0 = np.array([-150, -50])

# Interactive widget to link sliders to the update function
interactive_plot = interactive(rotate_screw, rotation_1=slider_rotations_1, rotation_2 = slider_rotations_2, rotation_3 = slider_rotations_3)
display(interactive_plot)

Button(description='Reset', style=ButtonStyle())

interactive(children=(IntSlider(value=0, description='rotation_1', max=50, min=-50), IntSlider(value=0, descri…

# To check how the equations of constrains look like, use this:

In [17]:
import sympy as sp

approx = True
change = 1
dist = 0.25

# Layer 1 points
p11x,p11z = sp.symbols('p^{11}_x , p^{11}_z', real=True) 
p11 = sp.Matrix([p11x,p11z,1])
p12x,p12z = sp.symbols('p^{12}_x , p^{12}_z', real=True)
p12 = sp.Matrix([p12x,p12z,1])
p13x,p13z = sp.symbols('p^{13}_x , p^{13}_z', real=True)
p13 = sp.Matrix([p13x,p13z,1])

# 0 Layer 2 points
p21x_0, p21z_0 = sp.symbols('p^{21_0}_x , p^{21_0}_z', real=True)
p21_0 = sp.Matrix([p21x_0,p21z_0,1])
p22x_0, p22z_0 = sp.symbols('p^{22_0}_x , p^{22_0}_z', real=True)
p22_0 = sp.Matrix([p22x_0,p22z_0,1])
p23x_0, p23z_0 = sp.symbols('p^{23_0}_x , p^{23_0}_z', real=True)
p23_0 = sp.Matrix([p23x_0,p23z_0,1])

# New Layer 2 points
p21x_i, p21z_i = sp.symbols('p^{21_i}_x , p^{21_i}_z', real=True)
p21_i = sp.Matrix([p21x_i,p21z_i,1])
p22x_i, p22z_i = sp.symbols('p^{22_i}_x , p^{22_i}_z', real=True)
p22_i = sp.Matrix([p22x_i,p22z_i,1])
p23x_i, p23z_i = sp.symbols(' p^{23_i}_x , p^{23_i}_z', real=True)
p23_i = sp.Matrix([p23x_i,p23z_i,1])

# Rotation Matrix
R_y,eta = sp.symbols('R_y,eta', real=True)

if approx:
	R_y = sp.Matrix([[1,eta,0],
					[-eta,1,0],
					[0, 0, 1],
					])
else:
	R_y = sp.Matrix([[sp.cos(eta),sp.sin(eta),0],
					[-sp.sin(eta),sp.cos(eta),0],
					[0, 0, 1],
					])

# translation vector
u,w = sp.symbols('u,w', real=True)
T = sp.Matrix([[1,0,u],
				[0,1,w],
				[0,0,1],
				])

# Can apply the same Transformation to all the points since its a rigid body transformation
p21_i = R_y* T *(p21_0)
p22_i = R_y* T *(p22_0)
p23_i = R_y* T *(p23_0)

values = {
	p11x:-75, p11z:60,
	p12x:-75, p12z:-60, 
	p13x:-150, p13z:55,
	p21x_0:30, p21z_0:60,
	p22x_0:30, p22z_0:-60,
	p23x_0:-150, p23z_0:-50
	}

#################
# CONSTRAINTS
#################
Eqs = []

d1_0 = (p11 - p21_0).norm()
d1_0 = d1_0.subs(values)
d1_i = (p11 - p21_i).norm()
d1_i = d1_i.subs(values)

d2_0 = (p12 - p22_0).norm()
d2_0 = d2_0.subs(values)
d2_i = (p12 - p22_i).norm()
d2_i = d2_i.subs(values)

d3_0 = (p13 - p23_0).norm()
d3_0 = d3_0.subs(values)
d3_i = (p13 - p23_i).norm()
d3_i = d3_i.subs(values)

if change == 1:
	Eq_d1 = sp.Eq(d1_0+dist,d1_i)
	Eq_d2 = sp.Eq(d2_0,d2_i)
	Eq_d3 = sp.Eq(d3_0,d3_i)

elif change == 2:
	Eq_d1 = sp.Eq(d1_0,d1_i)
	Eq_d2 = sp.Eq(d2_0+dist,d2_i)
	Eq_d3 = sp.Eq(d3_0,d3_i)


elif change == 3:
	Eq_d1 = sp.Eq(d1_0,d1_i)
	Eq_d2 = sp.Eq(d2_0,d2_i)
	Eq_d3 = sp.Eq(d3_0+dist,d3_i)

Eqs.extend([Eq_d1,Eq_d2, Eq_d3])

# Dot products between p2is are constant

d12_0 = (p21_0 - p22_0).norm()
d12_0 = d12_0.subs(values)
d12_i = (p21_i - p22_i).norm()
d12_i = d12_i.subs(values)
Eq_d12 = sp.Eq(d12_0,d12_i)
Eqs.append(Eq_d12)

d23_0 = (p22_0 - p23_0).norm()
d23_0 = d23_0.subs(values)
d23_i = (p22_i - p23_i).norm()
d23_i = d23_i.subs(values)
Eq_d23 = sp.Eq(d23_0,d23_i)
Eqs.append(Eq_d23)

d13_0 = (p21_0 - p23_0).norm()
d13_0 = d13_0.subs(values)
d13_i = (p21_i - p23_i).norm()
d13_i = d13_i.subs(values)	
Eq_d13 = sp.Eq(d13_0,d13_i)
Eqs.append(Eq_d13)

for eq in Eqs:
	display(eq)


Eq(105.25, sqrt((eta*u + 30*eta - w)**2 + (eta*w + 60*eta + u + 105)**2))

Eq(105, sqrt((eta*u + 30*eta - w)**2 + (eta*w - 60*eta + u + 105)**2))

Eq(105, sqrt((eta*w - 50*eta + u)**2 + (eta*u - 150*eta - w + 105)**2))

Eq(120, sqrt(14400*eta**2 + 14400))

Eq(50*sqrt(13), sqrt((180 - 10*eta)**2 + (180*eta + 10)**2))

Eq(10*sqrt(445), sqrt((110*eta + 180)**2 + (180*eta - 110)**2))

# Since there are no analytical solutions, we are solving it by numerical methods


In [18]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from ipywidgets import interactive
import ipywidgets as widgets

# Transformation function (now in 3D)
def transform(p, R, T):
    return np.dot(R, p) + T

# Rotation matrix for rotation in the XZ plane (affects only X and Z)
def rotation_matrix(eta):
    return np.array([[np.cos(eta), 0, np.sin(eta)],
                     [0, 1, 0],
                     [-np.sin(eta), 0, np.cos(eta)]])

def plot_layers(p21, p22, p23, eta=0, u=0, w=0, name='layer'):
    # Set up 3D plot
    fig = plt.figure(figsize=(10, 10), dpi=100)
    ax = fig.add_subplot(111, projection='3d')

    origin = np.array([0, 0, 0])  # Origin in 3D (x, y, z)
    solid_length = 400
    solid_width = 200

    # Layer 1 points (fixed in the XZ plane, y=0)
    p11 = np.array([-75, 0, 60])  # Add a z-coordinate for layer 1
    p12 = np.array([-75, 0, -60])
    p13 = np.array([-150, 0, 55])

    # Rectangle points in the XZ plane (fixed y = 0)
    rect_points = np.array([
        [-solid_length / 2, 0, -solid_width / 2],
        [solid_length / 2, 0, -solid_width / 2],
        [solid_length / 2, 0, solid_width / 2],
        [-solid_length / 2, 0, solid_width / 2],
        [-solid_length / 2, 0, -solid_width / 2]
    ])

    # Rotation and translation matrices
    R = rotation_matrix(eta)
    T = np.array([u, 0, w])  # Translation vector in 3D (X and Z axes)

    # Apply transformation to rectangle points
    transformed_rect_points = np.array([transform(p, R, T) for p in rect_points])
    original_rect_points = np.array([transform(p, np.eye(3), np.array([0, 0, 0])) for p in rect_points])

    # Create lower plate (solid color)
    lower_plate_points = np.array([[-solid_length / 2, 0, -solid_width / 2],
                                   [solid_length / 2, 0, -solid_width / 2],
                                   [solid_length / 2, 0, solid_width / 2],
                                   [-solid_length / 2, 0, solid_width / 2]])

    # Create the lower plate face as a solid color (set alpha = 1 for opacity)
    lower_face = Poly3DCollection([lower_plate_points], color='orange', linewidths=1, edgecolors='r', alpha=0.2)
    ax.add_collection3d(lower_face)

    # Create upper plate (transparent)
    upper_plate_points = transformed_rect_points
    upper_face = Poly3DCollection([upper_plate_points], color='blue', linewidths=1, edgecolors='b', alpha=0.2)
    ax.add_collection3d(upper_face)

    # Plot the points of layer 1 in the XZ plane (y=0)
    ax.scatter(p11[0], p11[2], color='yellow', label='p11')
    ax.text(p11[0], 0, p11[2], 'p11', fontsize=5)

    ax.scatter(p12[0], p12[2], color='green', label='p12')
    ax.text(p12[0], 0, p12[2], 'p12', fontsize=5)

    ax.scatter(p13[0], p13[2], color='blue', label='p13')
    ax.text(p13[0], 0, p13[2], 'p13', fontsize=5)

    # Plot the points of the transformed layer 2 in the XZ plane (y=0)
    ax.scatter(p21[0], p21[2], color='orange', label='p21')
    ax.text(p21[0], 0, p21[2], 'p21', fontsize=5)

    ax.scatter(p22[0], p22[2], color='purple', label='p22')
    ax.text(p22[0], 0, p22[2], 'p22', fontsize=5)

    ax.scatter(p23[0], p23[2], color='cyan', label='p23')
    ax.text(p23[0], 0, p23[2], 'p23', fontsize=5)

    ax.set_xlabel('X')
    ax.set_ylabel('Y')  # This will be 0 as we're working in the XZ plane
    ax.set_zlabel('Z')
    ax.set_title(f"3D Plot of Layers with Solid Lower Plate and Transparent Upper Plate")
    ax.legend()

     # Set the ticks on the axes to step by 100
    ax.set_xticks(np.arange(-300, 300, 100))  # X-axis ticks at intervals of 100
    ax.set_yticks(np.arange(-300, 300, 100))  # Y-axis ticks at intervals of 100
    ax.set_zticks(np.arange(-300, 300, 100))  # Z-axis ticks at intervals of 100


    # Adjust view angle to see from the top (Y as vertical)
    ax.view_init(elev=140, azim=90)  # Top-down view with Y axis vertical

    plt.show()

# Example function to solve and update plot
def rotate_screw(rotation_1, rotation_2, rotation_3):
    # Example points for layer 2 (modify as needed)
    p21 = np.array([30, 0, 60 + rotation_1])  # p21 in the XZ plane (y=0)
    p22 = np.array([30, 0, -60 + rotation_2])
    p23 = np.array([-150, 0, -50 + rotation_3])

    plot_layers(p21, p22, p23, eta=0, u=0, w=0)

# Interactive sliders to control screw rotations
slider_rotations_1 = widgets.FloatSlider(value=0, min=-13, max=13, step=1, description="Rotation 1")
slider_rotations_2 = widgets.FloatSlider(value=0, min=-13, max=13, step=1, description="Rotation 2")
slider_rotations_3 = widgets.FloatSlider(value=0, min=-13, max=13, step=1, description="Rotation 3")

interactive_plot = interactive(rotate_screw, rotation_1=slider_rotations_1, rotation_2=slider_rotations_2, rotation_3=slider_rotations_3)
display(interactive_plot)


interactive(children=(FloatSlider(value=0.0, description='Rotation 1', max=13.0, min=-13.0, step=1.0), FloatSl…

In [19]:
pip install pyvista jupyterlab
import pyvista as pv


SyntaxError: invalid syntax (41490371.py, line 1)